In [15]:
import tensorflow as tf
import numpy as np
import uuid
import matplotlib.pyplot as plt

#akhil: uncommented so that there is a random seed
#did the model not converge with this one? :(  
np.random.seed(2018)

#Declare hyperparameters
m = 1 # dimension
k_sq = 0.5
learning_rate = 0.01
epochs = 500
batch_size = 100
x_stddev = 3
activation_fn_1 = tf.nn.sigmoid
activation_fn = tf.nn.relu
num_units_1 = 150
num_units_2 = 30
decay = 1 - 1e-10

test_averaging = 100

In [17]:
x0 = tf.placeholder(tf.float32, [None, m])
z = tf.placeholder(tf.float32, [None, m])

l1 = tf.layers.dense(
  x0, num_units_1, activation=activation_fn_1, use_bias=True)
l2 = tf.layers.dense(
  l1, m, activation=activation_fn_1, use_bias=True)

u1 = l2
u1_cost = (tf.norm(u1)**2) / batch_size

# The observed value for the second controller is the original controlled with noise
x1 = x0 + u1
y1 = x1 + z

l3 = tf.layers.dense(
  y1, num_units_2, activation=activation_fn, use_bias=True)
l4 = tf.layers.dense(
  l3, m, activation=activation_fn, use_bias=True)

u2 = -l4
x2 = x1 + u2

u2_cost = (tf.norm(x2) ** 2) / batch_size
wits_cost = (k_sq * u1_cost) + u2_cost

adaptive_learning_rate = tf.placeholder_with_default(learning_rate, [])

optimizer = tf.train.GradientDescentOptimizer(learning_rate=adaptive_learning_rate).minimize(wits_cost)

init_op = tf.global_variables_initializer()

tf.summary.scalar("WITS_Cost", wits_cost)
merged_summary_op = tf.summary.merge_all()

with tf.Session() as sess:
    sess.run(init_op)
    uniq_id = "tmp/tensorboard-layers-api/" + uuid.uuid1().__str__()[:6]
    summary_writer = tf.summary.FileWriter(uniq_id, graph=tf.get_default_graph())
    x_train = np.random.normal(size=epochs * batch_size * m, scale=x_stddev)

  # Train for some epochs
    for step in range(epochs):
        x_batch = x_train[step: step + (batch_size * m)].reshape((batch_size, m))

        # Noise has variance 1
        z_batch = np.random.normal(size=(batch_size, m), scale=1)

        _, val, summary = sess.run(
        [optimizer, wits_cost, merged_summary_op],
        feed_dict={x0: x_batch, z: z_batch,
             adaptive_learning_rate: learning_rate * (decay**step)})

        if step % 50 == 0:
#         run_output = 'step:  ' + str(step) + ', value: ' + str(val)
#         f.write(run_output)
            print("step: {}, value: {}".format(step, val))
            summary_writer.add_summary(summary, step)

    # Test over a continuous range of X
#     x0_test = np.linspace(-2*x_stddev, 2*x_stddev, num=100)
#     u1_test, u2_test, x1_test = np.zeros((1, 100)), np.zeros((1, 100)), np.zeros((1, 100))

#     for i in range(100):
#         u1t, u2t, x1t = 0, 0, 0
#         for _ in range(test_averaging):
#             u1tmp, u2tmp, x1tmp = sess.run(
#                 [u1, u2, x1],
#                 feed_dict={x0: x0_test[i].reshape((1, 1)), z: np.random.normal(size=(1, 1), scale=1)})

#         u1t += u1tmp
#         u2t += u2tmp
#         x1t += x1tmp

#         u1_test[0, i] = u1t / test_averaging
#         u2_test[0, i] = -u2t / test_averaging
#         x1_test[0, i] = x1t / test_averaging

# l1, = plt.plot(x0_test, u1_test[0], label="U1 Test")
# l3, = plt.plot(x0_test, u2_test[0], label="U2 Test")
# plt.legend(handles=[l1, l3])
# plt.title("{} Unit NN With Activation Fn {}".format(
#   str(num_units_1), str(activation_fn)))
# plt.savefig("figs/figure_u_1.png")

# plt.clf()
# l2, = plt.plot(x0_test, x1_test[0], label="X1 Test")
# plt.title("{} Unit NN With Activation Fn {}".format(
#   str(num_units_1), str(activation_fn)))
# plt.legend(handles=[l2])
# plt.savefig("figs/figure_x_1.png")

InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_14' with dtype float and shape [?,1]
	 [[Node: Placeholder_14 = Placeholder[dtype=DT_FLOAT, shape=[?,1], _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'Placeholder_14', defined at:
  File "/Users/akhiljalan/anaconda/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/akhiljalan/anaconda/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/akhiljalan/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/akhiljalan/anaconda/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/akhiljalan/anaconda/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "/Users/akhiljalan/anaconda/lib/python3.6/site-packages/tornado/

In [3]:
x0_test

array([-6.        , -5.87878788, -5.75757576, -5.63636364, -5.51515152,
       -5.39393939, -5.27272727, -5.15151515, -5.03030303, -4.90909091,
       -4.78787879, -4.66666667, -4.54545455, -4.42424242, -4.3030303 ,
       -4.18181818, -4.06060606, -3.93939394, -3.81818182, -3.6969697 ,
       -3.57575758, -3.45454545, -3.33333333, -3.21212121, -3.09090909,
       -2.96969697, -2.84848485, -2.72727273, -2.60606061, -2.48484848,
       -2.36363636, -2.24242424, -2.12121212, -2.        , -1.87878788,
       -1.75757576, -1.63636364, -1.51515152, -1.39393939, -1.27272727,
       -1.15151515, -1.03030303, -0.90909091, -0.78787879, -0.66666667,
       -0.54545455, -0.42424242, -0.3030303 , -0.18181818, -0.06060606,
        0.06060606,  0.18181818,  0.3030303 ,  0.42424242,  0.54545455,
        0.66666667,  0.78787879,  0.90909091,  1.03030303,  1.15151515,
        1.27272727,  1.39393939,  1.51515152,  1.63636364,  1.75757576,
        1.87878788,  2.        ,  2.12121212,  2.24242424,  2.36

In [8]:
u1_test

array([[9.99975145e-01, 9.99970019e-01, 9.99967098e-01, 9.99961674e-01,
        9.99954283e-01, 9.99946654e-01, 9.99938428e-01, 9.99925673e-01,
        9.99913692e-01, 9.99895453e-01, 9.99876082e-01, 9.99849796e-01,
        9.99818861e-01, 9.99779344e-01, 9.99732494e-01, 9.99671340e-01,
        9.99593675e-01, 9.99494612e-01, 9.99367356e-01, 9.99206603e-01,
        9.98998582e-01, 9.98726368e-01, 9.98374343e-01, 9.97909606e-01,
        9.97299016e-01, 9.96489942e-01, 9.95412767e-01, 9.93972719e-01,
        9.92040634e-01, 9.89435732e-01, 9.85916734e-01, 9.81148362e-01,
        9.74677980e-01, 9.65894222e-01, 9.53989089e-01, 9.37919557e-01,
        9.16379690e-01, 8.87825549e-01, 8.50590825e-01, 8.03125739e-01,
        7.44430602e-01, 6.74602807e-01, 5.95348120e-01, 5.10142148e-01,
        4.23794985e-01, 3.41439754e-01, 2.67355412e-01, 2.04170927e-01,
        1.52704433e-01, 1.12340599e-01, 8.16177353e-02, 5.87617606e-02,
        4.20442149e-02, 2.99646631e-02, 2.13099439e-02, 1.514396

In [7]:
u2_test

array([[-0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
        -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
        -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
        -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
        -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
        -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
         2.23234110e-03, -0.00000000e+00, -0.00000000e+00,
        -0.00000000e+00,  3.56215588e-03,  5.64189686e-04,
         6.35662442e-03,  8.82912485e-04,  3.34948301e-03,
         2.23895302e-03,  8.27875268e-03,  9.78664774e-03,
         4.14471179e-02,  3.74349207e-02,  5.20114899e-02,
         4.15378436e-02,  4.91143018e-02,  4.24335897e-02,
         1.18470408e-01,  9.79649350e-02,  1.28080189e-01,
         1.39283717e-01,  2.07013085e-01,  1.23740979e-01,
         2.14241385e-01,  1.16928168e-01,  1.86641738e-01,
         1.61945611e-01,  2.66329527e-01,  2.56515294e-01,
         2.60838479e-01,  2.14508712e-01,  3.19975108e-0

In [9]:
x1_test

array([[-5.00002909, -4.87881517, -4.75760555, -4.6364069 , -4.51519728,
        -4.39398909, -4.27279472, -4.15158319, -4.0303936 , -3.90918946,
        -3.78800225, -3.66681194, -3.5456326 , -3.42446613, -3.30329537,
        -3.18214774, -3.06101441, -2.93989849, -2.81881428, -2.69776487,
        -2.57676339, -2.45582175, -2.3349607 , -2.21421409, -2.09361172,
        -1.97320652, -1.853073  , -1.7333014 , -1.61401951, -1.49541211,
        -1.37771749, -1.26127589, -1.14653575, -1.03410566, -0.92479765,
        -0.81965643, -0.71998525, -0.62732607, -0.54334939, -0.46960104,
        -0.40708503, -0.35570008, -0.31374356, -0.27773678, -0.24287164,
        -0.204015  , -0.15688688, -0.09885941, -0.02911369,  0.05173446,
         0.14222363,  0.24057975,  0.34507415,  0.4542073 ,  0.56676477,
         0.68181115,  0.79864502,  0.91675514,  1.03576839,  1.15542448,
         1.2755295 ,  1.39595723,  1.51661038,  1.63742077,  1.75834644,
         1.87935019,  2.00041318,  2.12152004,  2.2

In [10]:
summary_writer

# Everything Broke

Everything broke. Let's try running Nikunj's raw code.

In [6]:
import tensorflow as tf
import numpy as np
import uuid
import matplotlib.pyplot as plt

In [7]:
np.random.seed(2018)

#Declare hyperparameters
m = 1 # dimension
k_sq = 0.5
learning_rate = 0.01
epochs = 10000
batch_size = 100
x_stddev = 3
activation_fn_1 = tf.nn.sigmoid
activation_fn = tf.nn.relu
num_units_1 = 150
num_units_2 = 30
decay = 1 - 1e-10

test_averaging = 100

# Allow a batch of values to work
x0 = tf.placeholder(tf.float32, [None, m])
z = tf.placeholder(tf.float32, [None, m])

l1 = tf.layers.dense(
  x0, num_units_1, activation=activation_fn_1, use_bias=True)
l2 = tf.layers.dense(
  l1, m, activation=activation_fn_1, use_bias=True)

u1 = l2
u1_cost = (tf.norm(u1)**2) / batch_size

# The observed value for the second controller is the original controlled with noise
x1 = x0 + u1
y1 = x1 + z

l3 = tf.layers.dense(
  y1, num_units_2, activation=activation_fn, use_bias=True)
l4 = tf.layers.dense(
  l3, m, activation=activation_fn, use_bias=True)

u2 = -l4
x2 = x1 + u2

u2_cost = (tf.norm(x2) ** 2) / batch_size
wits_cost = (k_sq * u1_cost) + u2_cost

adaptive_learning_rate = tf.placeholder_with_default(learning_rate, [])

optimizer = tf.train.GradientDescentOptimizer(
  learning_rate=adaptive_learning_rate).minimize(wits_cost)

init_op = tf.global_variables_initializer()

tf.summary.scalar("WITS Cost", wits_cost)
merged_summary_op = tf.summary.merge_all()

with tf.Session() as sess:
  sess.run(init_op)
  uniq_id = "tmp/tensorboard-layers-api/" + uuid.uuid1().__str__()[:6]
  summary_writer = tf.summary.FileWriter(uniq_id, graph=tf.get_default_graph())
  x_train = np.random.normal(size=epochs * batch_size * m, scale=x_stddev)

  # Train for some epochs
  for step in range(epochs):
    x_batch = x_train[step: step + (batch_size * m)].reshape((batch_size, m))

    # Noise has variance 1
    z_batch = np.random.normal(size=(batch_size, m), scale=1)

    _, val, summary = sess.run(
      [optimizer, wits_cost, merged_summary_op],
      feed_dict={x0: x_batch, z: z_batch,
                 adaptive_learning_rate: learning_rate * (decay**step)})

    if step % 10 == 0:
      print("step: {}, value: {}".format(step, val))
      summary_writer.add_summary(summary, step)

  # Test over a continuous range of X
  x0_test = np.linspace(-2*x_stddev, 2*x_stddev, num=100)
  u1_test, u2_test, x1_test = np.zeros((1, 100)), np.zeros((1, 100)), np.zeros(
    (1, 100))

  for i in range(100):
    u1t, u2t, x1t = 0, 0, 0
    for _ in range(test_averaging):
      u1tmp, u2tmp, x1tmp = sess.run(
        [u1, u2, x1],
        feed_dict={x0: x0_test[i].reshape((1, 1)), z: np.random.normal(
          size=(1, 1), scale=1)})

      u1t += u1tmp
      u2t += u2tmp
      x1t += x1tmp

    u1_test[0, i] = u1t / test_averaging
    u2_test[0, i] = -u2t / test_averaging
    x1_test[0, i] = x1t / test_averaging

l1, = plt.plot(x0_test, u1_test[0], label="U1 Test")
l3, = plt.plot(x0_test, u2_test[0], label="U2 Test")
plt.legend(handles=[l1, l3])
plt.title("{} Unit NN With Activation Fn {}".format(
  str(num_units_1), str(activation_fn)))
plt.savefig("figs/figure_u_1.png")

plt.clf()
l2, = plt.plot(x0_test, x1_test[0], label="X1 Test")
plt.title("{} Unit NN With Activation Fn {}".format(
  str(num_units_1), str(activation_fn)))
plt.legend(handles=[l2])
plt.savefig("figs/figure_x_1.png")

INFO:tensorflow:Summary name WITS Cost is illegal; using WITS_Cost instead.


InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder' with dtype float and shape [?,1]
	 [[Node: Placeholder = Placeholder[dtype=DT_FLOAT, shape=[?,1], _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'Placeholder', defined at:
  File "/Users/akhiljalan/anaconda/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/akhiljalan/anaconda/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/akhiljalan/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/akhiljalan/anaconda/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/akhiljalan/anaconda/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "/Users/akhiljalan/anaconda/lib/python3.6/site-packages/tornado/platform/